In [ ]:
#@title Determine the Least-Common Keyword Using the Scryfall API

import requests, json, re, collections
from tqdm import tqdm  # For a progress bar

bulk_url = "https://api.scryfall.com/bulk-data/default_cards"
bulk_data_info = requests.get(bulk_url).json()
download_uri = bulk_data_info.get("download_uri")
print("Downloading default_cards bulk data from:")
print(download_uri)

cards = requests.get(download_uri).json()
print(f"Downloaded {len(cards)} cards (unique card names).")

https://data.scryfall.io/default-cards/default-cards-20250311090828.json
Downloaded 105029 cards (unique card names).


In [ ]:
# Step 2. Define the list of keyword abilities.
# (This list is based on Comprehensive Rules 702.2–702.176, with the following modifications:
#  • Ability words (such as Battalion, Bloodrush, etc.) have been removed.
#  • Evergreen keyword actions (such as Attach, Counter, Exile, Fight, Mill, Sacrifice, Scry, Tap/Untap)
#    have been removed.
#  • Extra keywords not in the comprehensive list have been added: Manifest, Meld, Radiation, Support, Surveil.
#  • The order of "Splice" and "Split Second" has been adjusted alphabetically.)
keywords = [
    "Deathtouch", "Defender", "Double Strike", "Enchant", "Equip", "First Strike", "Flash", "Flying",
    "Haste", "Hexproof", "Indestructible", "Intimidate", "Landwalk", "Lifelink", "Protection", "Reach",
    "Shroud", "Trample", "Vigilance", "Ward", "Banding", "Rampage", "Cumulative Upkeep", "Flanking",
    "Phasing", "Buyback", "Shadow", "Cycling", "Echo", "Horsemanship", "Fading", "Kicker", "Flashback",
    "Madness", "Fear", "Morph", "Amplify", "Provoke", "Storm", "Affinity", "Entwine", "Modular",
    "Sunburst", "Bushido", "Soulshift", "Splice", "Split Second", "Offering", "Ninjutsu", "Epic",
    "Convoke", "Dredge", "Transmute", "Bloodthirst", "Haunt", "Replicate", "Forecast", "Graft",
    "Recover", "Ripple", "Suspend", "Vanishing", "Absorb", "Aura Swap", "Delve", "Fortify",
    "Frenzy", "Gravestorm", "Poisonous", "Transfigure", "Champion", "Changeling", "Evoke", "Hideaway",
    "Prowl", "Reinforce", "Conspire", "Persist", "Wither", "Retrace", "Devour", "Exalted", "Unearth",
    "Cascade", "Annihilator", "Level Up", "Rebound", "Umbra Armor", "Infect", "Battle Cry",
    "Living Weapon", "Undying", "Miracle", "Soulbond", "Overload", "Scavenge", "Unleash", "Cipher",
    "Evolve", "Extort", "Fuse", "Bestow", "Tribute", "Dethrone", "Hidden Agenda", "Outlast", "Prowess",
    "Dash", "Exploit", "Menace", "Renown", "Awaken", "Devoid", "Ingest", "Myriad", "Surge", "Skulk",
    "Emerge", "Escalate", "Melee", "Crew", "Fabricate", "Partner", "Undaunted", "Improvise", "Aftermath",
    "Embalm", "Eternalize", "Afflict", "Ascend", "Assist", "Jump-Start", "Mentor", "Afterlife", "Riot",
    "Spectacle", "Escape", "Companion", "Mutate", "Encore", "Boast", "Foretell", "Demonstrate",
    "Daybound and Nightbound", "Disturb", "Decayed", "Cleave", "Training", "Compleated", "Reconfigure",
    "Blitz", "Casualty", "Enlist", "Read Ahead", "Ravenous", "Squad", "Space Sculptor", "Visit",
    "Prototype", "Living Metal", "More Than Meets the Eye", "For Mirrodin!", "Toxic", "Backup",
    "Bargain", "Craft", "Disguise", "Solved", "Plot", "Saddle", "Spree", "Freerunning", "Gift",
    "Offspring", "Impending",
    # Extra keywords that were missing from the Wikipedia source:
    "Manifest", "Meld", "Radiation", "Support", "Surveil"
]


regex_patterns = {}
for kw in keywords:
    # \b handles word boundaries
    pattern = r"\b" + re.escape(kw.lower()) + r"\b"
    regex_patterns[kw] = re.compile(pattern)

# Step 3. Count occurrences (per unique card) for each keyword.
keyword_counts = collections.defaultdict(int)

for card in tqdm(cards, desc="Processing cards"):
    # We combine texts from all faces of the card
    oracle_text = ""
    if "oracle_text" in card:
        oracle_text = card["oracle_text"]
    elif "card_faces" in card:
        oracle_text = " ".join(face.get("oracle_text", "") for face in card["card_faces"])
    text_lower = oracle_text.lower()
    for kw, pattern in regex_patterns.items():
        if pattern.search(text_lower):
            keyword_counts[kw] += 1

# Step 4. Determine the keyword with the minimum (nonzero) count.
nonzero_counts = {kw: count for kw, count in keyword_counts.items() if count > 0}
if not nonzero_counts:
    print("No keywords were found in any card!")
else:
    least_common_keyword = min(nonzero_counts, key=nonzero_counts.get)
    least_count = nonzero_counts[least_common_keyword]

    print("\nKeyword counts (only keywords that appear on at least one card):")
    for kw, count in sorted(nonzero_counts.items(), key=lambda x: x[1]):
        print(f"{kw}: {count}")

    print("\nThe least-common keyword is:")
    print(f"{least_common_keyword} (appears on {least_count} unique cards)")


Processing cards: 100%|██████████| 105029/105029 [01:02<00:00, 1681.51it/s]


Keyword counts (only keywords that appear on at least one card):
Transfigure: 1
Aura Swap: 1
Absorb: 2
Gravestorm: 3
Fortify: 4
Space Sculptor: 4
Radiation: 5
Frenzy: 6
Epic: 7
Poisonous: 8
Recover: 9
Ripple: 11
Amplify: 14
Ingest: 14
Tribute: 15
Hidden Agenda: 15
Undaunted: 18
Assist: 19
Offering: 20
Reinforce: 20
Meld: 21
Phasing: 22
Prowl: 23
Haunt: 23
Conspire: 24
Forecast: 24
Read Ahead: 25
Provoke: 25
Living Metal: 25
Melee: 26
Enlist: 26
Solved: 26
Ravenous: 27
Cipher: 28
Riot: 29
More Than Meets the Eye: 29
Demonstrate: 29
Spectacle: 31
Freerunning: 31
Afterlife: 31
Surge: 33
Dethrone: 33
Escalate: 34
Afflict: 35
Soulshift: 35
Craft: 35
Impending: 35
Squad: 35
Transmute: 37
Landwalk: 38
Graft: 38
Jump-Start: 38
Scavenge: 38
Awaken: 39
Battle Cry: 40
Sunburst: 40
Unleash: 40
Champion: 41
Spree: 42
Bargain: 42
Eternalize: 43
Outlast: 44
Rampage: 45
Boast: 46
Fabricate: 46
Prototype: 47
Casualty: 48
Training: 48
Offspring: 49
Renown: 49
Emerge: 49
Fuse: 50
Cleave: 51
Blitz: 52
Re

In [ ]:
cards_by_name = {}

for card in cards:
    name = card.get("name")
    oracle_text = ""
    if "oracle_text" in card:
        oracle_text = card["oracle_text"]
    elif "card_faces" in card:
        oracle_text = " ".join(face.get("oracle_text", "") for face in card["card_faces"])

    if name in cards_by_name:
        cards_by_name[name] += " " + oracle_text
    else:
        cards_by_name[name] = oracle_text

print(f"Unique card names: {len(cards_by_name)}")

# New keyword abilities. (I was lazy)
keywords = [
    "Deathtouch", "Defender", "Double Strike", "Enchant", "Equip", "First Strike", "Flash", "Flying",
    "Haste", "Hexproof", "Indestructible", "Intimidate", "Landwalk", "Lifelink", "Protection", "Reach",
    "Shroud", "Trample", "Vigilance", "Ward", "Banding", "Rampage", "Cumulative Upkeep", "Flanking",
    "Phasing", "Buyback", "Shadow", "Cycling", "Echo", "Horsemanship", "Fading", "Kicker", "Flashback",
    "Madness", "Fear", "Morph", "Amplify", "Provoke", "Storm", "Affinity", "Entwine", "Modular",
    "Sunburst", "Bushido", "Soulshift", "Splice", "Split Second", "Offering", "Ninjutsu", "Epic",
    "Convoke", "Dredge", "Transmute", "Bloodthirst", "Haunt", "Replicate", "Forecast", "Graft",
    "Recover", "Ripple", "Suspend", "Vanishing", "Absorb", "Aura Swap", "Delve", "Fortify",
    "Frenzy", "Gravestorm", "Poisonous", "Transfigure", "Champion", "Changeling", "Evoke", "Hideaway",
    "Prowl", "Reinforce", "Conspire", "Persist", "Wither", "Retrace", "Devour", "Exalted", "Unearth",
    "Cascade", "Annihilator", "Level Up", "Rebound", "Umbra Armor", "Infect", "Battle Cry",
    "Living Weapon", "Undying", "Miracle", "Soulbond", "Overload", "Scavenge", "Unleash", "Cipher",
    "Evolve", "Extort", "Fuse", "Bestow", "Tribute", "Dethrone", "Hidden Agenda", "Outlast", "Prowess",
    "Dash", "Exploit", "Menace", "Renown", "Awaken", "Devoid", "Ingest", "Myriad", "Surge", "Skulk",
    "Emerge", "Escalate", "Melee", "Crew", "Fabricate", "Partner", "Undaunted", "Improvise", "Aftermath",
    "Embalm", "Eternalize", "Afflict", "Ascend", "Assist", "Jump-Start", "Mentor", "Afterlife", "Riot",
    "Spectacle", "Escape", "Companion", "Mutate", "Encore", "Boast", "Foretell", "Demonstrate",
    "Daybound and Nightbound", "Disturb", "Decayed", "Cleave", "Training", "Compleated", "Reconfigure",
    "Blitz", "Casualty", "Enlist", "Read Ahead", "Ravenous", "Squad", "Space Sculptor", "Visit",
    "Prototype", "Living Metal", "More Than Meets the Eye", "For Mirrodin!", "Toxic", "Backup",
    "Bargain", "Craft", "Disguise", "Solved", "Plot", "Saddle", "Spree", "Freerunning", "Gift",
    "Offspring", "Impending",
    "Manifest", "Meld", "Radiation", "Support", "Surveil"
]

regex_patterns = {}
for kw in keywords:
    # \b handles word boundaries
    pattern = r"\b" + re.escape(kw.lower()) + r"\b"
    regex_patterns[kw] = re.compile(pattern)

keyword_counts = collections.defaultdict(int)
for name, oracle_text in tqdm(cards_by_name.items(), desc="Processing unique cards"):
    text_lower = oracle_text.lower()
    for kw, pattern in regex_patterns.items():
        if pattern.search(text_lower):
            keyword_counts[kw] += 1

nonzero_counts = {kw: count for kw, count in keyword_counts.items() if count > 0}
if not nonzero_counts:
    print("No keywords were found in any unique card!")
else:
    least_common_keyword = min(nonzero_counts, key=nonzero_counts.get)
    least_count = nonzero_counts[least_common_keyword]

    print("\nKeyword counts (per unique card name):")
    for kw, count in sorted(nonzero_counts.items(), key=lambda x: x[1]):
        print(f"{kw}: {count}")

    print("\nThe least-common keyword is:")
    print(f"{least_common_keyword} (appears on {least_count} unique cards)")


Unique card names: 33725


Processing unique cards: 100%|██████████| 33725/33725 [00:46<00:00, 725.00it/s] 


Keyword counts (per unique card name):
Transfigure: 1
Space Sculptor: 1
Aura Swap: 1
Absorb: 1
Fortify: 2
Gravestorm: 2
Radiation: 2
Poisonous: 3
Frenzy: 4
Epic: 5
Ripple: 6
Meld: 7
Impending: 7
Recover: 7
Undaunted: 7
Offering: 8
Escalate: 9
Amplify: 9
Dethrone: 10
Ingest: 10
Demonstrate: 10
Spectacle: 11
Read Ahead: 11
Tribute: 11
Reinforce: 11
Prowl: 12
Forecast: 12
Afterlife: 12
Conspire: 13
Melee: 13
Cleave: 13
Graft: 13
Compleated: 13
Outlast: 13
Haunt: 13
Provoke: 13
Living Metal: 13
Transmute: 14
Eternalize: 14
Hideaway: 14
Afflict: 14
Squad: 14
Hidden Agenda: 14
Freerunning: 15
Cipher: 15
Jump-Start: 15
Ravenous: 15
Enlist: 15
Solved: 15
Riot: 15
More Than Meets the Eye: 15
Training: 16
Dredge: 16
Surge: 16
Fabricate: 16
Emerge: 16
Extort: 17
Assist: 17
Awaken: 17
Battle Cry: 18
Phasing: 18
Embalm: 18
Sunburst: 18
Unleash: 18
Scavenge: 19
Umbra Armor: 20
Casualty: 20
Boast: 20
Living Weapon: 20
Rampage: 20
Decayed: 21
Spree: 21
Blitz: 21
Fading: 21
Support: 21
Reconfigure: 21

In [ ]:
#@title Search for a Keyword in Unique Cards

import re

# Ask for a keyword.
keyword_input = input("Enter a keyword to search for: ").strip()

if not keyword_input:
    print("No keyword entered. Exiting.")
else:
    # Create a regex pattern for whole-word matching (case-insensitive).
    pattern = re.compile(r"\b" + re.escape(keyword_input.lower()) + r"\b")

    matching_cards = []
    for name, oracle_text in cards_by_name.items():
        if pattern.search(oracle_text.lower()):
            matching_cards.append(name)

    print(f"\nFound {len(matching_cards)} unique cards with the keyword '{keyword_input}':\n")
    for card in matching_cards:
        print(card)


Enter a keyword to search for: radiation

Found 2 unique cards with the keyword 'radiation':

Strong, the Brutish Thespian
Aplan Mortarium


In [ ]:
#@title Print Matching Card Names for Selected Abilities

abilities_to_search = {
    "Transfigure": 1,
    "Space Sculptor": 1,
    "Aura Swap": 1,
    "Absorb": 1,
    "Radiation": 2
}

# Seach non-duplicates
for ability, expected in abilities_to_search.items():
    pattern = re.compile(r"\b" + re.escape(ability.lower()) + r"\b")
    matching_cards = [name for name, oracle_text in cards_by_name.items() if pattern.search(oracle_text.lower())]

    print("="*40)
    print(f"Ability: {ability}")
    print(f"Expected Count: {expected}, Found: {len(matching_cards)}")
    print("-"*40)
    if matching_cards:
        for card in matching_cards:
            print(card)
    else:
        print("No matching cards found.")
    print("="*40 + "\n")


Ability: Transfigure
Expected Count: 1, Found: 1
----------------------------------------
Fleshwrither

Ability: Space Sculptor
Expected Count: 1, Found: 1
----------------------------------------
Space Beleren

Ability: Aura Swap
Expected Count: 1, Found: 1
----------------------------------------
Arcanum Wings

Ability: Absorb
Expected Count: 1, Found: 1
----------------------------------------
Lymph Sliver

Ability: Radiation
Expected Count: 2, Found: 2
----------------------------------------
Strong, the Brutish Thespian
Aplan Mortarium



In [ ]:
#@title Print Full Description for Selected Cards

import requests

card_names = [
    "Fleshwrither",
    "Space Beleren",
    "Arcanum Wings",
    "Lymph Sliver",
    "Strong, the Brutish Thespian"
]

def print_card_details(card):
    if 'card_faces' in card:
        print(f"Name: {card.get('name')}")
        print(f"Mana Cost: {card.get('mana_cost', 'N/A')}")
        print(f"Type: {card.get('type_line')}")
        print("Card Faces:")
        for face in card["card_faces"]:
            print("  ---------------------")
            print(f"  Face Name: {face.get('name')}")
            print(f"  Mana Cost: {face.get('mana_cost', 'N/A')}")
            print(f"  Type: {face.get('type_line')}")
            print(f"  Oracle Text: {face.get('oracle_text', 'N/A')}")
            if face.get("flavor_text"):
                print(f"  Flavor Text: {face.get('flavor_text')}")
    else:
        print(f"Name: {card.get('name')}")
        print(f"Mana Cost: {card.get('mana_cost', 'N/A')}")
        print(f"Type: {card.get('type_line')}")
        print(f"Oracle Text: {card.get('oracle_text', 'N/A')}")
        if card.get("flavor_text"):
            print(f"Flavor Text: {card.get('flavor_text')}")

# Loop over each card name, and print.
for name in card_names:
    url = f"https://api.scryfall.com/cards/named?exact={name}"
    response = requests.get(url)
    if response.status_code != 200:
        print("="*60)
        print(f"Error fetching '{name}': {response.status_code}")
        print("="*60)
        continue

    card = response.json()
    print("="*60)
    print_card_details(card)
    print("="*60)
    print("\n")


Name: Fleshwrither
Mana Cost: {2}{B}{B}
Type: Creature — Horror
Oracle Text: Transfigure {1}{B}{B} ({1}{B}{B}, Sacrifice this creature: Search your library for a creature card with the same mana value as this creature, put that card onto the battlefield, then shuffle. Transfigure only as a sorcery.)


Name: Space Beleren
Mana Cost: {2}{W}{U}
Type: Legendary Planeswalker — Jace
Oracle Text: Space sculptor (Space Beleren divides the battlefield into alpha, beta, and gamma sectors. If a creature isn't assigned to a sector, its controller assigns it to one. Opponents assign first.)
+1: Creatures in each sector can be blocked this turn only by creatures in the same sector.
−1: Put a +1/+1 counter on each creature in the sector of your choice.
−5: Destroy all creatures in the sector of your choice.


Name: Arcanum Wings
Mana Cost: {1}{U}
Type: Enchantment — Aura
Oracle Text: Enchant creature
Enchanted creature has flying.
Aura swap {2}{U} ({2}{U}: Exchange this Aura with an Aura card in your